In [30]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os
import random
from sympy import mod_inverse
from PIL import Image as im

%matplotlib inline
mod_inverse?
def get_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    print(image.shape)
    return image

In [2]:
path = r'C:\Users\soule\OneDrive\Pictures\addie.jpg'
l = get_image(path)
rows,cols = l.shape
print(l)
#data = im.fromarray(l.astype(np.uint8)) 
#data.show() 

(450, 600)
[[ 30  31  33 ...  71  79  95]
 [ 31  32  34 ...  76  74  99]
 [ 31  32  34 ... 105  87 113]
 ...
 [ 90  88  85 ... 116  98  91]
 [ 87  89  78 ... 111 120 101]
 [ 66  63  48 ... 110 129 101]]


In [3]:
def IntegerDigits(p,b):
    arr = []
    while p>=1:
        arr.append(p%b)
        p = p//b
    A = arr[::-1]
    return len(A)-1

def IntegerToArray(n,base):
    A = []
    while n>=1:
        A.append(n%base)
        n = n//base
    Aflip = A[::-1]
    return Aflip

def singleLargeInteger(A,base):
    integer = 0
    power = 0
    Anew = A[::-1]
    for i in range(len(Anew)):
        integer += (Anew[i] * base**i)
    return integer

def Generate_Point(a,b,p):
    x = random.randint(0,p)
    y = np.arange(0,p)
    possibilites = []
    for i in range(len(y)):
        yval = ((x**3) + (a*x) + b)%p
            #print(x[i],y[j],y[j]**2 % p, yval)
            #print(yval)
        if (((y[i])**2)%p == yval):
            possibilites.append(y[i])
    if(len(possibilites)==0):
        return Generate_Point(a,b,p)
    else:
        yindex = random.randint(0,len(possibilites)-1)
        return x, possibilites[yindex]               

In [4]:
totalPixs = rows*cols
print(totalPixs)
groupSize = IntegerDigits(totalPixs,256)
groupedArray = np.reshape(l, (totalPixs//groupSize, groupSize), order='C')

270000


In [5]:
Pm = []
for i in range(totalPixs//groupSize):
    newValue = singleLargeInteger(groupedArray[i],256)
    Pm.append(newValue)
print(len(Pm))
if len(Pm)%2 == 1:
    Pm.append(0)
#print(Pm)
#print(maxP)

135000


In [6]:
imagePointsArray = np.reshape(Pm, (len(Pm)//2, 2), order='C')
print(imagePointsArray)

[[ 7711  8484]
 [10025 11052]
 [11306  9762]
 ...
 [20035 13882]
 [13119 21852]
 [24942 33125]]


In [7]:
p = 524287
a = 345257
b = -93473


In [8]:
#G = random point
G = Generate_Point(a,b,p)
print(G)

(88822, 193839)


In [9]:
private_a = 741
private_A = Generate_Point(a,b,p)
print(private_A)

(281371, 344720)


In [10]:
private_b = 9812
private_B = Generate_Point(a,b,p)
print(private_B)

(347307, 509674)


In [34]:
def point_addition(x1,y1,x2,y2,p,a):
    x3,y3=0,0
    if x1!=x2 and y1!=y2:
        den = mod_inverse(x2-x1,p)
        print(den)
        m = ((y2-y1)*den)%p
        print(m)
        x3 = (m**2 - x1 - x2)%p
        print(x3)
        y3 = (m*(x1-x3) - y1)%p
        print(y3)
        
    elif x1==x2 and y1==y2:
        m = ((3*(x1**2) + a)*(mod_inverse(2*y1,p)))%p
        print(m)
        x3 = (m**2 - 2*x1)%p
        print(x3)
        y3 = (m*(x1-x3) - y1)%p
        print(y3)
    return x3,y3

def point_subtraction(x1,y1,x2,y2,p,a):
    y2 = -(y2)
    y2 = y2%p
    return point_addition(x1,y1,x2,y2,p,a)

def point_multiplication(x,y,k,p,a):
    xnew,ynew = x,y
    for i in range(k-1):
        xnew,ynew = point_addition(x,y,xnew,ynew,p,a)
    return xnew,ynew

In [36]:
G_plus_A1 = point_addition(G[0],G[1],private_A[0],private_A[1],p,a)

#A1 = point_multiplication(G_plus_A[0],G_plus_A[1],private_a,p,a)
#A2 = point_multiplication(private_A[0],private_A[1],private_a,p,a)

480770
142647
176986
97185


<ipython-input-34-da1de96797b3>:6: RuntimeWarning: overflow encountered in long_scalars
  m = ((y2-y1)*den)%p
<ipython-input-34-da1de96797b3>:10: RuntimeWarning: overflow encountered in long_scalars
  y3 = (m*(x1-x3) - y1)%p


In [37]:
2**32

4294967296

In [13]:
G_plus_B = point_addition(G[0],G[1],private_B[0],private_B[1],p,a)

B1 = point_multiplication(G_plus_B[0],G_plus_B[1],private_b,p,a)
B2 = point_multiplication(private_B[0],private_B[1],private_b,p,a)

<ipython-input-11-a5a030b238fe>:4: RuntimeWarning: overflow encountered in long_scalars
  m = ((y2-y1)*mod_inverse(x2-x1,p))%p
<ipython-input-11-a5a030b238fe>:7: RuntimeWarning: overflow encountered in long_scalars
  y3 = m*(x1-x3) - y1
<ipython-input-11-a5a030b238fe>:11: RuntimeWarning: overflow encountered in long_scalars
  m = ((3*(x1**2) + a)*(mod_inverse(2*y1,p)))%p
<ipython-input-11-a5a030b238fe>:13: RuntimeWarning: overflow encountered in long_scalars
  y3 = (m*(x1-x3) - y1)%p


In [14]:
A_B = point_multiplication(B2[0],B2[1],private_a,p,a)
B_A = point_multiplication(A2[0],A2[1],private_b,p,a)

In [15]:
b_plus_one_A1 = point_multiplication(A1[0],A1[1],(private_b+1),p,a)

b_plus_one_A1_minus_A2 = point_subtraction(b_plus_one_A1[0],b_plus_one_A1[1],A2[0],A2[1],p,a)

added_to_M = point_addition(b_plus_one_A1_minus_A2[0],b_plus_one_A1_minus_A2[1],A_B[0],A_B[1],p,a)

<ipython-input-11-a5a030b238fe>:11: RuntimeWarning: overflow encountered in long_scalars
  m = ((3*(x1**2) + a)*(mod_inverse(2*y1,p)))%p
<ipython-input-11-a5a030b238fe>:13: RuntimeWarning: overflow encountered in long_scalars
  y3 = (m*(x1-x3) - y1)%p
<ipython-input-11-a5a030b238fe>:4: RuntimeWarning: overflow encountered in long_scalars
  m = ((y2-y1)*mod_inverse(x2-x1,p))%p
<ipython-input-11-a5a030b238fe>:7: RuntimeWarning: overflow encountered in long_scalars
  y3 = m*(x1-x3) - y1


In [16]:
coordinates, variables = imagePointsArray.shape
encrypted_array = []
print(coordinates)
r=coordinates
for i in range(coordinates):
    EP = point_addition(imagePointsArray[i][0],imagePointsArray[i][1],added_to_M[0],added_to_M[1],p,a)
    encrypted_array.append(EP[0])
    encrypted_array.append(EP[1])

67500


In [17]:
cipher_array = []
#print(cipher_array[0])
for i in range(len(encrypted_array)):
    arr = IntegerToArray(encrypted_array[i],256)
    while len(arr)>2:
        arr.remove(arr[0])
    while len(arr)<2:
        arr.insert(0,0)
    for j in arr:
        cipher_array.append(j)
print(len(cipher_array))

270000


In [18]:
encryption_arr = np.array(cipher_array, dtype=int)
encryption_arr = np.reshape(encryption_arr, (rows, cols), order='C')
encryption_arr

array([[238, 189, 201, ..., 254, 140, 155],
       [ 65, 135, 111, ..., 200,   3,  32],
       [ 65, 135, 111, ..., 191, 219, 166],
       ...,
       [ 92, 113, 212, ..., 168, 206,  55],
       [ 26, 128,  84, ...,  32, 244, 128],
       [  2,  94,   1, ..., 177, 220, 216]])

In [19]:
e_data = im.fromarray(encryption_arr.astype(np.uint8)) 
e_data.show() 

In [20]:
#Decryption
data = np.asarray(e_data)
rows,cols = data.shape
print(data.shape)
print(data)

(450, 600)
[[238 189 201 ... 254 140 155]
 [ 65 135 111 ... 200   3  32]
 [ 65 135 111 ... 191 219 166]
 ...
 [ 92 113 212 ... 168 206  55]
 [ 26 128  84 ...  32 244 128]
 [  2  94   1 ... 177 220 216]]


In [21]:
totalPixs = rows*cols
print(totalPixs)
groupSize = IntegerDigits(totalPixs,256)
groupedArray = np.reshape(data, (totalPixs//groupSize, groupSize), order='C')

270000


In [22]:
Em = []
for i in range(totalPixs//groupSize):
    newValue = singleLargeInteger(groupedArray[i],256)
    Em.append(newValue)
print(len(Em))
if len(Em)%2 == 1:
    Em.append(0)
#print(Em)

135000


In [23]:
EmPointsArray = np.reshape(Em, (len(Em)//2, 2), order='C')
print(EmPointsArray)

[[61117 51517]
 [38728 36383]
 [45260 38121]
 ...
 [61621  1097]
 [57150 21630]
 [ 2993 56536]]


In [24]:
G_plus_B1 = point_addition(G[0],G[1],B1[0],B1[1],p,a)
private_a_G_plus_B1 = point_multiplication(G_plus_B1[0],G_plus_B1[1],private_a,p,a)
private_a_G_plus_B1_plus_BA = point_addition(private_a_G_plus_B1[0],private_a_G_plus_B1[1],B_A[0],B_A[1],p,a)

<ipython-input-11-a5a030b238fe>:4: RuntimeWarning: overflow encountered in long_scalars
  m = ((y2-y1)*mod_inverse(x2-x1,p))%p
<ipython-input-11-a5a030b238fe>:7: RuntimeWarning: overflow encountered in long_scalars
  y3 = m*(x1-x3) - y1
<ipython-input-11-a5a030b238fe>:11: RuntimeWarning: overflow encountered in long_scalars
  m = ((3*(x1**2) + a)*(mod_inverse(2*y1,p)))%p
<ipython-input-11-a5a030b238fe>:13: RuntimeWarning: overflow encountered in long_scalars
  y3 = (m*(x1-x3) - y1)%p


In [25]:
coordinates, variables = EmPointsArray.shape
decrypted_array = []
print(coordinates)
r=coordinates
for i in range(coordinates):
    DP = point_subtraction(EmPointsArray[i][0],EmPointsArray[i][1],private_a_G_plus_B1_plus_BA[0],private_a_G_plus_B1_plus_BA[1],p,a)
    decrypted_array.append(DP[0])
    decrypted_array.append(DP[1])
#print(decrypted_array)

67500


In [26]:
plaintext_array = []
#print(plaintext_array[0])
for i in range(len(decrypted_array)):
    arr = IntegerToArray(decrypted_array[i],256)
    while len(arr)>2:
        arr.remove(arr[0])
    while len(arr)<2:
        arr.insert(0,0)
    for j in arr:
        plaintext_array.append(j)
#print(plaintext_array)

In [27]:
decryption_arr = np.array(plaintext_array, dtype=int)
decryption_arr = np.reshape(decryption_arr, (rows, cols), order='C')
decryption_arr

array([[146, 136, 244, ...,  63, 157, 162],
       [231, 195, 153, ..., 140, 117,  75],
       [231, 195, 153, ...,   1, 215,  21],
       ...,
       [254, 246,  87, ...,  39, 205, 157],
       [ 47, 203, 123, ..., 117, 161, 185],
       [ 29,  21, 183, ..., 231, 235,  20]])

In [28]:
d_data = im.fromarray(decryption_arr.astype(np.uint8)) 
d_data.show() 